In [ ]:
%reset -f
import time
import pickle
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import derivatives as d
from chameleon import Chameleon

In [ ]:
n_elems = 500
steps = 1000
chameleon = Chameleon(E=10, n_elems=n_elems)

In [ ]:
# active_stress = 0.1*np.linspace(1, 10, chameleon.n_elems)
active_stress = 0.001*np.ones(chameleon.n_elems)
# active_stress = .002*np.random.random_sample(chameleon.n_elems) + .001
active_stress

## Is this is too slow? In order to do RL I may need to be able to simulate much faster than this. (Jax?)

In [ ]:
%%time
chameleon.step(active_stress, sim_steps=steps)

In [ ]:
for i in tqdm(range(50)):
    chameleon = Chameleon(E=10, n_elems=n_elems)
    # active_stress = .0002*np.random.random_sample(chameleon.n_elems) + .0001
    chameleon.step(active_stress, sim_steps=steps)
    

In [ ]:
active_stress

In [ ]:
chameleon.pos_f

In [ ]:
np.diff(chameleon.pos_f)

## There seems to be an issue when the stress is not monotonically increasing. The second element often ends up behind the first and as a result the sim crashes. I am not exactly sure why it is happening but it is clearly related to the boundary condition being imposed on the first element. If I then try to make sure the second element is always beyond the first, the third element goes behind the second. I think the issue is related to the stress on the first element causing a high force since the 0->1 element gets stretched a lot and it may be less than the stress from the 1->2 element. If the active stress is not enough to overcome that, then the second element would move to the left and maybe end up behind the first element. 

Actually i don't buy that argument because then a uniform stress might also cause this issue. In that case we would have a large stress from the stretching 0->1 element and if the active stress wasn't enough to overcome it we could end up with the same issue. But whenever I run uniform simulations I don't see that problem (no matter the magnitude of the active stress).

One way to debug is to save the NET force (and component forces maybe) on each element at each point in time and to see what happens. 

# I think its the Derivative of $\sigma$ that's important here.

## Why do the elements come in after the first time step? 

Then after the first time step is over they move out as expected. Strange. It's only the element on the end and it comes from satisfying the boundary condition.

In [ ]:
chameleon.position_history[0]

In [ ]:
chameleon.position_history[1]

In [ ]:
chameleon.position_history[2]

# some hacky code to store the position history for later visualization.


In [ ]:
history = []
for i in range(len(chameleon.position_history)):
    x_p = chameleon.position_history[i]
    y = np.zeros(chameleon.n_elems)
    ar = np.vstack((x_p, y)).T
    history.append(ar)

with open("history.txt", "wb") as fp:
    pickle.dump(history, fp)